## Loading Dataset

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('covid_toy.csv')
df.sample(5)

,age,gender,fever,cough,city,has_covid
87,47,Male,101.0,Strong,Bangalore,No
34,74,Male,102.0,Mild,Mumbai,Yes
75,5,Male,102.0,Mild,Kolkata,Yes
92,82,Female,102.0,Strong,Kolkata,No
12,25,Female,99.0,Strong,Kolkata,No


In [ ]:
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


In [ ]:
df['city'].value_counts()

,count
city,
Kolkata,32
Bangalore,30
Delhi,22
Mumbai,16


In [ ]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


<p>We need to apply Ordinal encoding on 'cough'<p/>
<p>We need to apply one hot encoding on 'gender' and 'city'<p/>
<p>We need to apply Simple Imputer, to handle missing values in 'fever'<p/>

## Train Test split

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['has_covid'])
Y = df['has_covid']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Traditonal approach without column transformer

In [ ]:
# Applying Simple Imputer on fever column
from sklearn.impute import SimpleImputer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [ ]:
# Applying ordinal encoding on cough column
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [ ]:
# Applying One hot encoding on gender and city column
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

In [ ]:
# Extracting age column
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [ ]:
# Combining all the transfomed columns and age column
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)
X_train_transformed.shape

(80, 7)

##Approach By using column transformer

In [ ]:
# Specifying our column transformer
from sklearn.compose import ColumnTransformer

# All the transformers are passed in the transformers list inside a tuple
# The transformer tuple entry is of following format :-
#  ("tranformer_name", transformer_object(), ['col_name1', 'col_name2])
# remainder=passthrough, means to ignore those columns where we don't want to apply transformer
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city'])
], remainder='passthrough')

In [ ]:
# Applying our specified tranformer on x_train and X_test
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)
X_train_transformed.shape

(80, 7)